In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from itertools import product

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)



In [ ]:

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


# CIFAR-10 class names
class_names = [
    "Dog", "Frog", "Horse", "Ship", "Truck",
    "Airplane", "Automobile", "Bird", "Cat", "Deer"
]

# Function to plot and save CIFAR-10 images
def save_cifar10_visualization(images, labels, class_names, num_images=16, filename="cifar10_sample.png"):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(4, 4, i + 1)
        plt.imshow(images[i])
        plt.title(class_names[labels[i][0]])
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(filename)
    print(f"Visualization saved as {filename}")

# Save the visualization
save_cifar10_visualization(x_train, y_train, class_names, num_images=16, filename="cifar10_sample.png")


In [ ]:
# Load and preprocess CIFAR-10 dataset
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# Hyperparameter options
learning_rates = [0.05, 0.0001]
kernel_sizes = [(3, 3), (7, 7)]
batch_norm_options = [True, False]
batch_sizes = [16, 256]


In [ ]:
# Function to build and train the model
def build_and_train_model(learning_rate, kernel_size, batch_norm, batch_size):
    model = models.Sequential()

    # First convolutional layer
    model.add(layers.Conv2D(32, kernel_size, padding='same', activation='relu', input_shape=(32, 32, 3)))
    if batch_norm:
        model.add(layers.BatchNormalization())

    # Second convolutional layer
    model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    if batch_norm:
        model.add(layers.BatchNormalization())

    # First max pooling and dropout
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))

    # Third and fourth convolutional layers
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    if batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
    if batch_norm:
        model.add(layers.BatchNormalization())

    # Second max pooling and dropout
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    # Fifth and sixth convolutional layers
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    if batch_norm:
        model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    if batch_norm:
        model.add(layers.BatchNormalization())

    # Third max pooling and dropout
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.4))

    # Fully connected layer
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(x_train, y_train, epochs=15, batch_size=batch_size, validation_data=(x_test, y_test), verbose=0)

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return test_accuracy

In [ ]:
# Iterate through all combinations of hyperparameters
results = []
for lr, ks, bn, bs in product(learning_rates, kernel_sizes, batch_norm_options, batch_sizes):
    print(f"Training with LR={lr}, Kernel={ks}, BatchNorm={bn}, BatchSize={bs}...")
    accuracy = build_and_train_model(lr, ks, bn, bs)
    results.append((lr, ks, bn, bs, accuracy))
    print(f"Test Accuracy: {accuracy:.4f}")

# Find and display the best accuracy
best_result = max(results, key=lambda x: x[4])  # x[4] is the accuracy
print("\nBest Configuration:")
print(f"Learning Rate: {best_result[0]}")
print(f"Kernel Size: {best_result[1]}")
print(f"Batch Normalization: {best_result[2]}")
print(f"Batch Size: {best_result[3]}")
print(f"Best Accuracy: {best_result[4]:.4f}")